# Infection Rate vs Public Announcements and Events
> Looking at the comparison of Government announcements/Public events on case loads.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/events.png

In [1]:
#hide

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
import altair as alt

from IPython.display import HTML, display
import pytz

import ipywidgets as widgets

from datetime import datetime, timedelta, timezone

print('Libraries successfully imported.')

Libraries successfully imported.


In [2]:
#hide
#%%time

url = 'https://data.ontario.ca/dataset/f4112442-bdc8-45d2-be3c-12efae72fb27/resource/455fd63b-603d-4608-8216-7d8647f43350/download/conposcovidloc.csv'
url_2 = 'https://data.ontario.ca/dataset/f4f86e54-872d-43f8-8a86-3892fd3cb5e6/resource/ed270bb8-340b-41f9-a7c6-e8ef587e6d11/download/covidtesting.csv'
url_3 = 'https://data.ontario.ca/dataset/f4f86e54-872d-43f8-8a86-3892fd3cb5e6/resource/8a88fe6d-d8fb-41a3-9d04-f0550a44999f/download/daily_change_in_cases_by_phu.csv'

#geo = gpd.read_file('https://data.ontario.ca/dataset/f4112442-bdc8-45d2-be3c-12efae72fb27/resource/4f39b02b-47fe-4e66-95b6-e6da879c6910/download/conposcovidloc.geojson') 

src_conpos = pd.read_csv(url, index_col=0, parse_dates=['Accurate_Episode_Date', 'Case_Reported_Date', 'Test_Reported_Date', 'Specimen_Date']).reset_index()
src_testing = pd.read_csv(url_2, index_col=0, parse_dates=['Reported Date']).reset_index()
src_daily = pd.read_csv(url_3, index_col=0, parse_dates=['Date']).reset_index()

# duplicate entry for December 5th, 2020 causing error
src_testing.drop_duplicates(inplace=True)

print('Source files successfully loaded.')

Source files successfully loaded.


In [3]:
#hide

df_testing = src_testing.drop(['Confirmed Negative',	'Presumptive Negative',	'Presumptive Positive',
                  'Total patients approved for testing as of Reporting Date',
                  'Under Investigation'], axis=1).fillna(0)

df_testing['New Cases'] = df_testing['Total Cases'].fillna(0).diff()
df_testing['New Deaths'] = df_testing['Deaths'].fillna(0).diff()
df_testing['Completed Tests Cumulative'] = df_testing['Total tests completed in the last day'].cumsum()

# rolling averages
numdays = 7

df_testing['New Cases 5 Day Average'] = round(df_testing['New Cases'].rolling(5).mean())
df_testing['New Cases 7 Day Average'] = round(df_testing['New Cases'].rolling(numdays).mean())
df_testing['New Tests 7 Day Average'] = round(df_testing['Total tests completed in the last day'].rolling(numdays).mean())

df_testing['7 Day Avg Percent Positivity'] = round((df_testing['New Cases 7 Day Average'] / df_testing['New Tests 7 Day Average']) *100, 2)

df_testing

,Reported Date,Confirmed Positive,Resolved,Deaths,Total Cases,Total tests completed in the last day,Percent positive tests in last day,Number of patients hospitalized with COVID-19,Number of patients in ICU with COVID-19,Number of patients in ICU on a ventilator with COVID-19,Total Positive LTC Resident Cases,Total Positive LTC HCW Cases,Total LTC Resident Deaths,Total LTC HCW Deaths,New Cases,New Deaths,Completed Tests Cumulative,New Cases 5 Day Average,New Cases 7 Day Average,New Tests 7 Day Average,7 Day Avg Percent Positivity
0,2020-01-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,2020-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,2020-01-28,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
3,2020-01-30,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
4,2020-01-31,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,2021-02-02,17451.0,247236.0,6238.0,270925.0,28552.0,4.6,1192.0,341.0,253.0,14586.0,6218.0,3618.0,10.0,745.0,14.0,9619346.0,1692.0,1746.0,50965.0,3.43
363,2021-02-03,16811.0,248981.0,6305.0,272097.0,52418.0,3.3,1066.0,336.0,254.0,14607.0,6250.0,3647.0,10.0,1172.0,67.0,9671764.0,1559.0,1675.0,50568.0,3.31
364,2021-02-04,16330.0,250937.0,6393.0,273660.0,64467.0,2.6,1101.0,323.0,241.0,14672.0,6300.0,3683.0,10.0,1563.0,88.0,9736231.0,1459.0,1600.0,50540.0,3.17
365,2021-02-05,15722.0,253170.0,6438.0,275330.0,62710.0,2.5,1043.0,325.0,225.0,14702.0,6341.0,3697.0,10.0,1670.0,45.0,9798941.0,1424.0,1576.0,49636.0,3.18


In [4]:
#hide

# todo: webscrape Canadian website for holidays and dates for automation

public_events = [
{"name": "Easter Weekend", "date": "2020-04-10"},
{"name": "Mother's Day", "date": "2020-05-10", "ay" : 60},
{"name": "Victoria Day", "date": "2020-05-18"},
{"name": "Trinity Bellwoods<br>Park Weekend", "date": "2020-05-23", "ay": -100, "ax": 80},
{"name": "Father's Day", "date": "2020-06-21", "ay": 40},
{"name": "Canada Day", "date": "2020-07-01"},
{"name": "Civic Holiday", "date": "2020-08-03"},
{"name": "Schools reopen<br>for 2020-21", "date": "2020-09-08"},
{"name": "Thanksgiving<br>Weekend", "date": "2020-10-12", "ax": -40, "ay": -50},
{"name": "Halloween", "date": "2020-10-31", "ax": -20},
{"name": "Diwali", "date": "2020-11-14", "ax": -60, "ay": -25},
{"name": "Black Friday", "date": "2020-11-27", "ax": -20},
{"name": "Vaccine Rollout", "date": "2020-12-15", "ax": -30},
{"name": "School closures<br>until Jan 15", "date": "2020-12-21", "ay": 80, "ax": 40},
{"name": "Christmas", "date": "2020-12-25", "ax": -20},
{"name": "Boxing Week", "date": "2020-12-26", "ax": 80},
{"name": "New Year's Day", "date": "2021-01-01"},
]

government_events = [
{"name": "Ontario Lockdown #1", "date": "2020-03-23","ax": -20, "ay": -80},
#{"name": "Some regions enter stage 2", "date": "2020-06-08"},
{"name": "Ontario in Stage 2", "date": "2020-06-24", "ay": -40},
{"name": "Ontario in Stage 3", "date": "2020-07-31", "ay": -30},
{"name": "Schools reopen<br>for 2020-21", "date": "2020-09-08", "ax": -40, "ay": -80},
{"name": "Second Wave confirmed", "date": "2020-09-28", "ax": -40, "ay": -80},
{"name": "Peel/Toronto/Ottawa<br>revert to<br>modified Stage 2", "date": "2020-10-12","ax":30, "ay": 100},
{"name": "Peel/Ottawa<br>in Orange Zone", "date": "2020-11-07","ax": 25, "ay": 60},
{"name": "Peel/Toronto<br>in Red Zone", "date": "2020-11-12", "ax": -80, "ay": -30},
{"name": "Halton in Red Zone", "date": "2020-11-16", "ax": -50},
{"name": "Peel/Toronto in<br>Lockdown<br>#2", "date": "2020-11-23", "ay": -80},
{"name": "York/Windsor<br>in Lockdown<br>#2", "date": "2020-12-14", "ay": 80},
{"name": "Vaccine Rollout", "date": "2020-12-15", "ay": -100},
{"name": "School closures<br>until Jan 25", "date": "2020-12-21", "ay": 80, "ax": 50}
]

other_events = [
{"name": "GoodLife email request to lobby MPPs", "date": "2020-10-28"},          
]

# Sources:
# https://www.peelschools.org/schools/reopening/Pages/default.aspx
# https://globalnews.ca/news/6859636/ontario-coronavirus-timeline/
# https://news.ontario.ca/en/release/59790/ontario-announces-provincewide-shutdown-to-stop-spread-of-covid-19-and-save-lives

In [5]:
#hide

# append events to an annotation dictionary

public_dates_annot = []

for event in public_events:
      if event["date"]:
        ax = event.get("ax")
        if ax is None:
            ax = 0
        ay = event.get("ay")
        if ay is None:
            ay = -60
        public_dates_annot.append(dict(
                  x=event["date"],
                  y=df_testing[df_testing["Reported Date"] == event["date"]]["New Cases 5 Day Average"].values[0],
                  xref="x",
                  yref="y",
                  text=event["name"],
                  showarrow=True,
                  arrowhead=3,
                  ax=ax,
                  ay=ay
        ))

government_dates_annot = []

for event in government_events:
      if event["date"]:
        ax = event.get("ax")
        if ax is None:
            ax = 0
        ay = event.get("ay")
        if ay is None:
            ay = -60
        government_dates_annot.append(dict(
                  x=event["date"],
                  y=df_testing[df_testing["Reported Date"] == event["date"]]["New Cases 5 Day Average"].values[0],
                  xref="x",
                  yref="y",
                  text=event["name"],
                  showarrow=True,
                  arrowhead=3,
                  ax=ax,
                  ay=ay
        ))

government_dates_annot[0]

{'arrowhead': 3,
 'ax': -20,
 'ay': -80,
 'showarrow': True,
 'text': 'Ontario Lockdown #1',
 'x': '2020-03-23',
 'xref': 'x',
 'y': 58.0,
 'yref': 'y'}

In [6]:
#hide_input

fig_public = go.Figure()
fig_public.add_trace(go.Scatter(x=df_testing['Reported Date'],
                               y=df_testing['New Cases 5 Day Average'],
                               mode="lines",
                               #line=dict(color="maroon", dash="dot"),
                               name='5d Rolling Avg',
                               showlegend=True
))

fig_public.update_layout(annotations=public_dates_annot,
                        title="Infection Rate Compared to Key Public Events",
                        template='ggplot2',
                        width=1000,
                        xaxis_title="Reported Date",
                        yaxis_title="Confirmed Cases")

fig_public.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,))

In [7]:
#hide_input

fig_gov = go.Figure()
fig_gov.add_trace(go.Scatter(x=df_testing['Reported Date'],
                             y=df_testing['New Cases 5 Day Average'],
                             mode="lines",
                             #line=dict(color="blue", dash="dot"),
                             showlegend=True,
                             name='5d Rolling Avg',
 
))

fig_gov.update_layout(annotations=government_dates_annot,
                        title="Infection Rate Compared to Governement Announcements",
                        template='seaborn',
                        width=1000,
                        xaxis_title="Reported Date",
                        yaxis_title="Confirmed Cases",)

fig_gov.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,))